# Relation between opEE and mutual information

Density matrix $\rho$ suppots on disjoint areas $A$ and $B$.

The mutual information $I(A:B)$ is defined to be $I(A:B) = S(A)+S(B)-S(AB)$.

Given operator basis $\{a_i\}$ for area $A$ and $\{b_i\}$ for area $B$. $\rho$ can be represented as $\rho=\sum_{i,j}c_{ij}a_i\otimes b_j$. Asume $\{s_k\}$ are singular values of normalized matrix $c_{ij}/||c_{ij}||_2$, then the operator entanglement entropy of $\rho$ between area A and B is given by $OpEE(\rho)= -\sum_i s_i^2 \log s_i^2 $

In this notebook we try to check numerically whether $I(A:B)=OpEE(\rho)$ holds in general.

## Setup

The density matrix can be expanded as:
$$\rho = \sum_{ijkl} \alpha_{i,j,k,l}|i\rangle_A \langle j|\otimes |k\rangle_B \langle l|$$
or just a tensor $\alpha_{ijkl}$

The reduced density matrices for A and B are: $\alpha_{iikl}$ and $\alpha_{ijkk}$ (Einstein notation addapted).

To get the $c_{ij}$ appeared above, we need to merge $i, j$ (and $k,l$) into a single index. A simple way is defining $I = i*d_A + j$ and $J = k*d_B + l$. Then $\alpha_{ijkl} = c_{IJ}$ and is ready for the SVD.

## Code

In [64]:
import numpy as np


In [107]:
d_A = 4
d_B = 8

# sample a positive definite matrix as density matrix

cc = lambda x: np.moveaxis(x, [0,1,2,3], [1,0,3,2]).conj()
tmp  = np.random.randn(d_A, d_A, d_B, d_B) + np.random.randn(d_A, d_A, d_B, d_B)*1j
tmp = tmp + cc(tmp)

# tmp2[i, j, k, l] = tmp[i, m, k, n] * tmp[m, j, n,l]

tmp2 = np.zeros((d_A, d_A, d_B, d_B), dtype=np.complex128)
for i in range(d_A):
    for j in range(d_A):
        for k in range(d_B):
            for l in range(d_B):
                for m in range(d_A):
                    for n in range(d_B):
                        tmp2[i, j, k, l] += tmp[i, m, k, n] * tmp[m, j, n,l]

# L1 normalize
trace = 0
for i in range(d_A):
    for j in range(d_B):
        trace += tmp2[i, i, j, j]
rho = tmp2/trace

In [108]:
rhoAB = np.zeros((d_A * d_B, d_A * d_B), dtype=np.complex128)
rhoA = np.zeros((d_A, d_A), dtype=np.complex128)
rhoB = np.zeros((d_B, d_B), dtype=np.complex128)
vecAB = np.zeros((d_A * d_A, d_B * d_B), dtype=np.complex128)

for i in range(d_A):
    for j in range(d_A):
        for k in range(d_B):
            for l in range(d_B):
                rhoAB[i * d_B + k, j * d_B + l] = rho[i, j, k, l]
                vecAB[i * d_A + j, k * d_B + l] = rho[i, j, k, l]
                if k==l:
                    rhoA[i, j] += rho[i, j, k, l]
                if i==j:
                    rhoB[k, l] += rho[i, j, k, l]

vecAB /= np.linalg.norm(vecAB)

In [182]:
S = lambda a: (-a*np.log(a)).sum()
Sa = lambda a, alpha: np.log((a**alpha).sum())/(1-alpha)


S_AB = S(np.linalg.eigvals(rhoAB).real)
S_A = S(np.linalg.eigvals(rhoA).real)
S_B = S(np.linalg.eigvals(rhoB).real)
I_AB = S_A + S_B - S_AB

sv = np.linalg.svd(vecAB, compute_uv=False)
opEE = S(sv**2)

opEE = Sa(sv**2, 2)



In [183]:
I_AB

0.4347394613716733

In [184]:
opEE

1.1220755903637725

In [112]:
opEE/I_AB

4.22965541339702